In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    "cifar10",
    split = ["train", "test"],
    shuffle_files = True,
    as_supervised = True,
    with_info = True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteX6X6V7/cifar10-train.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.2.incompleteX6X6V7/cifar10-test.tfrecord


  0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [3]:
def normalize_image(img, label):
  return tf.cast(img, tf.float32)/255.0, label

from tensorflow.python.ops.image_ops_impl import random_flip_left_right
def augment(image, label):
  new_height = new_width = 32
  image = tf.image.resize(image, (new_height, new_width))

  # Grayscale - probabalistic

  if tf.random.uniform((), minval=0, maxval=1) < 0.1:
    image = tf.tile(tf.image.rgb_to_grayscale(image), [1,1,3])
  
  image = tf.image.random_brightness(image, max_delta = 0.1)
  image = tf.image.random_contrast(image, lower = 0.1, upper =0.2)

  # Flipping - Careful, don't destroy label
  image = tf.image.random_flip_left_right(image) #50%
  # image = tf.image.random.flip_up_down(image) #50%

  return image, label

In [4]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds_train = ds_train.map(normalize_image, num_parallel_calls=AUTO).cache().shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(augment, num_parallel_calls=AUTO).batch(BATCH_SIZE).prefetch(AUTO)
ds_test = ds_test.map(normalize_image, num_parallel_calls=AUTO).batch(BATCH_SIZE).prefetch(AUTO)

In [5]:
class CNNBlock(layers.Layer):
  def __init__(self, channels):
    super().__init__()
    self.cnn = layers.Conv2D(channels, 3, padding='same', activation='leaky_relu', kernel_regularizer=keras.regularizers.l2(0.001))
    self.batchNorm = layers.BatchNormalization()
    # self.pool = layers.MaxPooling2D()
  
  def call(self, x, training = False):
    x = self.cnn(x)
    x = self.batchNorm(x)
    return tf.nn.relu(x)

class resBlock(layers.Layer):
  def __init__(self, channels):
    super().__init__()
    self.cnn1 = CNNBlock(channels[0])
    self.cnn2 = CNNBlock(channels[1])
    self.cnn3 = CNNBlock(channels[2])
    self.pool = layers.MaxPooling2D()
    self.identity_map = layers.Conv2D(channels[1], 1)
  
  def call(self, x, training = False):
    out = self.cnn1(x, training=training)
    out = self.cnn2(out, training=training)
    out = self.cnn3(self.identity_map(x) + out, training=training)
    return self.pool(out)

In [7]:
model = keras.Sequential([
    keras.Input(shape=(32, 32,3)),
    
    resBlock([32,32,64]),
    # resBlock([64, 128, 128]),
    # resBlock([128, 256, 256]),

    layers.Flatten(),
    layers.Dense(64, activation = "leaky_relu"),
    layers.Dropout(0.3),
    layers.Dense(10)
])

# model = keras.Sequential([
#     keras.Input(shape=(32,32,3)),
#     ResNet_lite(10)
# ])
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(3e-4),
    metrics = ["accuracy"]
)

## Callback - checkpoint

In [15]:
save_callback = keras.callbacks.ModelCheckpoint(
    'checkpoint/',
    save_weights_only = True,
    monitor = 'accuracy',
    save_best_only = False
)

In [17]:
def scheduler(epoch, lr):
  if epoch<2:
    return lr
  
  else:
    return lr*0.99

lr_scheduler = keras.callbacks.LearningRateScheduler(
    scheduler,
    verbose = 1
)

In [26]:
class CustomCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if logs.get("accuracy") > 0.85:
      print("Acc over 80% stopping training")
      self.model_stop_training = True

## Compile and Fit

In [27]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(3e-4),
    metrics = ["accuracy"]
)

In [29]:
model.fit(ds_train, epochs=5, verbose=2, callbacks= [save_callback, lr_scheduler, CustomCallback()])


Epoch 1: LearningRateScheduler setting learning rate to 0.0002910897310357541.
Epoch 1/5
1563/1563 - 9s - loss: 0.4707 - accuracy: 0.8497 - lr: 2.9109e-04 - 9s/epoch - 6ms/step

Epoch 2: LearningRateScheduler setting learning rate to 0.0002910897310357541.
Epoch 2/5
Acc over 80% stopping training
1563/1563 - 9s - loss: 0.4622 - accuracy: 0.8523 - lr: 2.9109e-04 - 9s/epoch - 6ms/step

Epoch 3: LearningRateScheduler setting learning rate to 0.00028817883372539655.
Epoch 3/5
Acc over 80% stopping training
1563/1563 - 9s - loss: 0.4536 - accuracy: 0.8562 - lr: 2.8818e-04 - 9s/epoch - 6ms/step

Epoch 4: LearningRateScheduler setting learning rate to 0.00028529703587992116.
Epoch 4/5
Acc over 80% stopping training
1563/1563 - 9s - loss: 0.4416 - accuracy: 0.8611 - lr: 2.8530e-04 - 9s/epoch - 6ms/step

Epoch 5: LearningRateScheduler setting learning rate to 0.0002824440796393901.
Epoch 5/5
Acc over 80% stopping training
1563/1563 - 9s - loss: 0.4315 - accuracy: 0.8641 - lr: 2.8244e-04 - 9s/e

In [20]:
model_2 = keras.Sequential([
    keras.Input(shape=(32, 32,3)),
    
    resBlock([32,32,64]),
    # resBlock([64, 128, 128]),
    # resBlock([128, 256, 256]),

    layers.Flatten(),
    layers.Dense(64, activation = "leaky_relu"),
    layers.Dropout(0.3),
    layers.Dense(10)
])

# model = keras.Sequential([
#     keras.Input(shape=(32,32,3)),
#     ResNet_lite(10)
# ])
model_2.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = keras.optimizers.Adam(3e-4),
    metrics = ["accuracy"]
)

In [30]:
model_2.load_weights('checkpoint/')

In [31]:
model_2.evaluate(ds_test, verbose=2)

313/313 - 1s - loss: 2.4258 - accuracy: 0.5787 - 1s/epoch - 4ms/step


[2.425813913345337, 0.5787000060081482]

In [32]:
model.evaluate(ds_test, verbose=2)

313/313 - 1s - loss: 4.2431 - accuracy: 0.5787 - 1s/epoch - 4ms/step


[4.243082046508789, 0.5787000060081482]